<a href="https://colab.research.google.com/github/lilnoes/notebooks/blob/main/horse_or_human_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
# !pip install --upgrade tensorflow
# !pip install tensorflow-gpu
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
from tensorflow import data as data
import numpy as np
print(tf.__version__)

2.1.0


In [ ]:
path_to_file = keras.utils.get_file('shakespear.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
def gen_series():
  i = 0
  while True:
    size = np.random.randint(1,10)
    yield i, tf.random.uniform((size,), minval=0, maxval=10 ,dtype=tf.int32)
    i += 1

In [ ]:
for i,val in gen_series():
  if i == 5:
    break
  print(val)

tf.Tensor([7 5 2 8 9 9 0 9], shape=(8,), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor([5 8 6 7 9 9 9 4 2], shape=(9,), dtype=int32)
tf.Tensor([0 6], shape=(2,), dtype=int32)
tf.Tensor([8 6 3 4 2 5 5], shape=(7,), dtype=int32)


In [ ]:
dataset = data.Dataset.from_generator(gen_series, (tf.int32, tf.int32), ((),(None,)))
dataset = dataset.shuffle(100).take(20)
for i in dataset:
  print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=75>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 5, 8, 1], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=92>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 1], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 8, 0, 7, 5], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=57>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([7, 7, 4, 3], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=60>, <tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 8, 0, 4], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=36>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([6, 3], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=41>, <tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 0, 2, 4, 7], dtype=int32)>)
(<tf.Tensor: shape=(), dtype=int32, numpy=12>, <tf.Tensor: shape=(5,), dtype=int32, numpy=array([3, 8, 8, 6, 7], dtype=int32)

In [ ]:
dataset = data.Dataset.from_tensor_slices([1,2,3,4,5,6,7,8])
# dataset.take(3).reduce(0, lambda v,x: v+x)


<TensorSliceDataset shapes: (), types: tf.int32>

In [ ]:
train_dataset , test_dataset = dataset['train'], dataset['test']
print(train_dataset.element_spec)

(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


In [ ]:
def printExamples(ds, numbers=2):
  for x, y in ds.take(numbers):
    print(x[0].shape)
    print(x[1].shape)

In [ ]:
encoder = info.features['text'].encoder
print(encoder)

<SubwordTextEncoder vocab_size=8185>


In [ ]:
BUFFER = 10000
BATCH = 64

In [ ]:
ps = ((None,), ())
train_ds = train_dataset.shuffle(BUFFER).padded_batch(BATCH, ps)
test_ds = test_dataset.padded_batch(BATCH, ps)

In [ ]:
printExamples(train_ds)

(1230,)
(1230,)
(848,)
(848,)


In [ ]:
E_DIM = 64
inputs = Input( (None,) )
embedding = Embedding(encoder.vocab_size, E_DIM)(inputs)
rnn1 = Bidirectional(LSTM(E_DIM))(embedding)
dense1 = Dense(E_DIM, activation='relu')(rnn1)
preds = Dense(1, activation='sigmoid')(dense1)

In [ ]:
model = keras.models.Model(inputs, preds)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_ds, epochs=10, validation_data=test_ds, validation_steps=30)

Epoch 1/10
391/391 [==============================] - 49s 126ms/step - loss: 0.3517 - accuracy: 0.8596 - val_loss: 0.4557 - val_accuracy: 0.8021
Epoch 2/10
391/391 [==============================] - 47s 121ms/step - loss: 0.2746 - accuracy: 0.8960 - val_loss: 0.3783 - val_accuracy: 0.8422
Epoch 3/10
391/391 [==============================] - 47s 121ms/step - loss: 0.2642 - accuracy: 0.8972 - val_loss: 0.4315 - val_accuracy: 0.8427
Epoch 4/10
391/391 [==============================] - 47s 121ms/step - loss: 0.2935 - accuracy: 0.8797 - val_loss: 0.5159 - val_accuracy: 0.7521
Epoch 5/10
391/391 [==============================] - 47s 121ms/step - loss: 0.2111 - accuracy: 0.9207 - val_loss: 0.4280 - val_accuracy: 0.8438
Epoch 6/10
391/391 [==============================] - 47s 121ms/step - loss: 0.1664 - accuracy: 0.9401 - val_loss: 0.5676 - val_accuracy: 0.7714
Epoch 7/10
391/391 [==============================] - 48s 123ms/step - loss: 0.1960 - accuracy: 0.9272 - val_loss: 0.5133 - val_ac

In [ ]:
dmodel.save('/content/drive/My Drive/ml/trained models/horse-or-human')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/ml/trained models/horse-or-human/assets
